In [1]:
import jijmodeling as jm
import numpy as np
import ommx

def create_problem():
    """
    Create the JijModeling problem definition.
    """
    # Define placeholders for model parameters
    msize = jm.Placeholder("msize", description="Size of the square matrix (n x n)")
    scale = jm.Placeholder("scale", description="Scaling factor for the decomposition")

    # Define sets and parameter placeholders
    J = jm.Placeholder("J", ndim=1, description="Index set for matrix rows and columns")
    A_mn = jm.Placeholder("A", ndim=2, description="Target matrix to decompose (scaled doubly stochastic)")
    P_i = jm.Placeholder("P", ndim=3, description="Set of 3D permutation matrices (|I| x msize x msize)")
    I = jm.Placeholder("I", ndim=1, description="Index set for permutations")

    # Define decision variables
    x = jm.IntegerVar("x", shape=(I.shape[0],), lower_bound=0, upper_bound=scale,
                    description="Integer weights for each permutation matrix")
    z = jm.BinaryVar("z", shape=(I.shape[0],),
                    description="Binary activation variable for each permutation matrix")

    # Create the optimization problem
    problem = jm.Problem("Birkhoff Integer Decomposition", sense=jm.ProblemSense.MINIMIZE)

    # Define index element i
    i = jm.Element("i", belong_to=I)

    # Objective: minimize the number of permutation matrices used (i.e., number of active z[i])
    problem += jm.sum(i, z[i])

    # Constraint c1: sum of all weights equals the given scale
    problem += jm.Constraint("c1", jm.sum(i, x[i]) == scale)

    # Constraint c2: for all matrix elements (m,n), the weighted sum of permutation matrices equals the target matrix a3
    # This enforces A = sum_i x[i] * P_i[i]
    m = jm.Element("m", belong_to=J)
    n = jm.Element("n", belong_to=J)
    problem += jm.Constraint("c2", jm.sum(i, x[i] * P_i[i, m, n]) == A_mn[m, n], forall=[m, n])

    # Constraint c3: x[i] > 0 implies z[i] = 1 (linking integer variable with binary)
    # Enforces: x[i] <= scale * z[i]
    problem += jm.Constraint("c3", x[i] <= scale * z[i], forall=[i])

    return problem

In [2]:
problem= create_problem()
problem

In [3]:
def permutation_to_matrix(perm):
    n= len(perm)
    matrix =[[0]*n for _ in range(n)]
    for i in range(n):
        matrix[i][perm[i]-1]=1
    return matrix
def convert_all_permutations(perms):
    return[permutation_to_matrix(p) for p in perms]

In [14]:
import numpy as np
instance_data = {
    "msize": 3,
    "scale": 1000,
    "J": np.arange(0, 3),
    "A": np.array([[916, 84, 0], [84, 457, 459], [0, 459, 541]]),
    "P": convert_all_permutations(np.array([[1, 2, 3], [1, 3, 2], [2, 1, 3], [2, 3, 1], [3, 1, 2], [3, 2, 1]])),
    "I": np.arange(0, 6) #{1,...,6}
}
instance_data

{'msize': 3,
 'scale': 1000,
 'J': array([0, 1, 2]),
 'A': array([[916,  84,   0],
        [ 84, 457, 459],
        [  0, 459, 541]]),
 'P': [[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
  [[1, 0, 0], [0, 0, 1], [0, 1, 0]],
  [[0, 1, 0], [1, 0, 0], [0, 0, 1]],
  [[0, 1, 0], [0, 0, 1], [1, 0, 0]],
  [[0, 0, 1], [1, 0, 0], [0, 1, 0]],
  [[0, 0, 1], [0, 1, 0], [1, 0, 0]]],
 'I': array([0, 1, 2, 3, 4, 5])}

In [4]:
instance_data = {
    "msize": 3,
    "scale": 1000,
    "J": np.arange(0, 3),
    "A": np.array([[0, 492, 508],[ 636, 315, 49],[ 364, 193, 443]]),
    "P": convert_all_permutations(np.array([
      [2, 1, 3],[ 3, 1, 2],[ 2, 1, 3],[ 3, 2, 1],[ 2, 1, 3],[ 2, 1, 3],[ 2, 3, 1], [2, 3, 1],[ 3,
      1, 2]
    ])),
    "I": np.arange(0, 9) #{1,...,msize!}
}
instance_data

{'msize': 3,
 'scale': 1000,
 'J': array([0, 1, 2]),
 'A': array([[  0, 492, 508],
        [636, 315,  49],
        [364, 193, 443]]),
 'P': [[[0, 1, 0], [1, 0, 0], [0, 0, 1]],
  [[0, 0, 1], [1, 0, 0], [0, 1, 0]],
  [[0, 1, 0], [1, 0, 0], [0, 0, 1]],
  [[0, 0, 1], [0, 1, 0], [1, 0, 0]],
  [[0, 1, 0], [1, 0, 0], [0, 0, 1]],
  [[0, 1, 0], [1, 0, 0], [0, 0, 1]],
  [[0, 1, 0], [0, 0, 1], [1, 0, 0]],
  [[0, 1, 0], [0, 0, 1], [1, 0, 0]],
  [[0, 0, 1], [1, 0, 0], [0, 1, 0]]],
 'I': array([0, 1, 2, 3, 4, 5, 6, 7, 8])}

In [5]:
ommx.v1.Instance = jm.Interpreter(instance_data).eval_problem(problem)

In [6]:
ommx.v1.Instance.objective

Function(x0 + x1 + x2 + x3 + x4 + x5 + x6 + x7 + x8)

In [7]:
import ommx_pyscipopt_adapter as scip_ad
solution = scip_ad.OMMXPySCIPOptAdapter.solve(ommx.v1.Instance)
print(f"{solution.objective=}, {solution.feasible=}")

solution.objective=4.0, solution.feasible=True
